<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

**Иcходные данные**

Необходимо защитить данные данные клиентов страховой компании. 

Нам предоставлены данные клиентов: пол, возраст и зарплата застрахованного, количество членов его семьи, количество страховых выплат клиенту за последние 5 лет. 

**Цель исследования**

Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При преобразовании качество моделей машинного обучения не должно ухудшиться.

**Ход исследования**

1. Обзор и подготовка данных.   
2. Поиск и обоснование решения.
3. Разработка алгоритма преобразования данных для решения задачи.
4. Обучение и проверка модели.

## Обзор данных

### Загрузка данных

Импортируем библиотеки.

In [1]:
import numpy as np
import os
import pandas as pd
import random as rnd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Загрузим и посмотрим данные.

In [2]:
pth1 = 'insurance.csv'
pth2 = '/datasets/insurance.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Файл не найден')

In [3]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Таблица содержит 5 признаков, один из которых целевой. Общее число объекстов 5000. Явные пропуски отсутствуют.

Согласно документации к данным: 

**Признаки**
* `Пол` — пол клиента
* `Возраст` — возраст клиента
* `Зарплата` — доход клиента
* `Члены семьи` — количество людей в семье, кроме клиента

**Целевой признак**:
* `Страховые выплаты` - количество страховых выплат клиенту за последние 5 лет.

По результату первичного обзора данных можем сделать следующие выводы:
1. Наименования всех признаков не соответствуют зминому стилю.
2. Явные пропуски в данных отсутствуют.
3. Отсутствуют категориальные признаки (пол преобразован в бинарный).

Нам необходимо:
1. Переименовать признаки.
2. Проверить данные на дубликаты.
3. Разделить данные на выборки.
4. Масштабировать признаки.

### Предобработка данных


Ззменим названия признаков, приведем их в соответствие стилю.

In [4]:
df.columns = ['sex', 'age', 'salary', 'family_members', 'insurance_payments']

# проверим результат
df.head(2)

,sex,age,salary,family_members,insurance_payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1


Проверим данные на дубликаты.

In [5]:
df.duplicated().sum()

153

В данных обнаружено 153 дублеката. Однако в исходной таблицы тсутствуют идентификаторы клиентов. Возможно у клиентв просто одинаковый надор признаков, в таком случае удаление дубликатов может исказить статистику. Оставим дублекаты в выборке.

Проверим все данные на аномалии в виде отрицательных значений.

In [6]:
df[df < 0].count()

sex                   0
age                   0
salary                0
family_members        0
insurance_payments    0
dtype: int64

Признаки `age` и `salary`, судя по первичному обзору, можем перевести в int. Проверим это.

In [7]:
df[['age_rem', 'salary_rem']] = df[['age', 'salary']] % 1
print(f"Переменных типа float в столбце 'age': {df.loc[df['age_rem'] !=0, 'age_rem'].count()} из {df['age_rem'].shape[0]}")
print(f"Переменных типа float в столбце 'salary': {df.loc[df['salary_rem'] !=0, 'salary_rem'].count()} из {df['salary_rem'].shape[0]}")

#посмотрим максимальное и минимальное значение 'salary'
print(f"Маскимальное значение'salary': {df['salary'].max()}. Минимальное значение'salary': {df['salary'].min()}")

Переменных типа float в столбце 'age': 0 из 5000
Переменных типа float в столбце 'salary': 40 из 5000
Маскимальное значение'salary': 79000.0. Минимальное значение'salary': 5300.0


Дробную часть имеет мене 1% данных в столбце salary. Учитывая, что сами значения столбца имеет порядок тысяч и десятков тысяч, столь малой частью можем принебречь. Переведем столбцы `age` и `salary` в формат uint8 и uint32. За одно изменим типы инт в оставльных столбцах на int8.

In [8]:
list = ['age', 'sex', 'family_members', 'insurance_payments']
for col in list:
    df[col] = np.uint8(df[col])

df['salary'] = np.uint32(df['salary'])

# удалим вспомогательные столбцы
df = df.drop(columns=['age_rem', 'salary_rem'])

# проверим результат
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sex                 5000 non-null   uint8 
 1   age                 5000 non-null   uint8 
 2   salary              5000 non-null   uint32
 3   family_members      5000 non-null   uint8 
 4   insurance_payments  5000 non-null   uint8 
dtypes: uint32(1), uint8(4)
memory usage: 39.2 KB


Разделим данные на выборки.
Выделим целивой признак.

In [9]:
df_features = df.drop('insurance_payments', axis=1)
df_target = df['insurance_payments']

Поскольку подбор параметров для лучшей модели не требуется, валидационную выборку можно опустить.

In [10]:
features_train, features_test, target_train, target_test = train_test_split(df_features, df_target, test_size=0.25, random_state=777)

Проверим размер выборок.

In [11]:
print(f'Тестовая выборка составляет {features_test.shape[0] / df_features.shape[0]:.0%} от общей.')
print(f'Обучающая выборка составляет {features_train.shape[0] / df_features.shape[0]:.0%} от общей.')

Тестовая выборка составляет 25% от общей.
Обучающая выборка составляет 75% от общей.


В таблице присутствуют признаки с разными разбросами значений. Для обучения модели линейной регрессии масштабируем признаки.

In [12]:
# обозначим признаки
scaler_features = df_features.columns

# обучим объект для масштабирования
scaler = StandardScaler()
scaler.fit(features_train[scaler_features])

# создадим переменные для отмасштабированных данных
features_train_sc = features_train
features_test_sc = features_test

# отключим предупреждение
pd.options.mode.chained_assignment = None

# масштабируем
features_train_sc[scaler_features] = scaler.transform(features_train[scaler_features])
features_test_sc[scaler_features] = scaler.transform(features_test[scaler_features])

# проверим результат
features_test_sc.head()

,sex,age,salary,family_members
3437,0.988862,-0.107347,0.513023,1.644896
1646,0.988862,-0.700381,0.955340,-0.187925
4035,0.988862,-1.412023,0.603497,-1.104336
2041,-1.011263,0.011260,-0.210768,-0.187925
1920,-1.011263,0.960115,1.397656,-0.187925


Масштабирование выполнено успешно.

## Умножение матриц

Ответим на вопрос и обоснуем решение.

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
* Изменится. Приведите примеры матриц.
* Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** Не изменится.

**Обоснование:** 

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $A$ — обратимая матрица, высота которой равна количеству признаков

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Сначала рассмотрим базовую формулу.

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Тогда а равно:

$$
a = X(X^T X)^{-1} X^T y
$$

В случае домножения Х на обратимую матрицу А:

Предсказания:

$$
a = XAw
$$

Задача обучения:

$$
w = \arg\min_w MSE(XAw, y)
$$

Формула обучения:

$$
w = ((XA)^T XA)^{-1} (XA)^T y
$$

В таком случае:

$$
a = XA ((XA)^T XA)^{-1} (XA)^T y
$$

Преобразуем транспорирование произведения матриц в произведение транспорированных матриц:

$$
a = XA ((A^T X^T XA)^{-1} A^T X^T y
$$

В скобках получаем произведение 3 квадратных матриц $ A^T, X^T X, A $. Можем преобразовать обратную матрицу произведения в произведение обратных матриц:

$$
a = XA (A)^{-1}(X^TX)^{-1}(A^T)^{-1} A^T X^T y
$$

Преобразуем произведения $A (A)^{-1}$ и $(A^T)^{-1} A^T$ в $E$:

$$
a = XE(X^TX)^{-1}E X^T y
$$

Поскольку, произведение любой матрицы на единичную матрицу равно этой матрице, $E$ можем опустить:

$$
a = X(X^TX)^{-1}X^T y
$$

То есть мы получили исходную формулу.

**Вывод:** Таким образом получаем, что домножение на обратимую матрицу не меняет уравнение.

## Алгоритм преобразования

**Алгоритм**

Создадим функцию шифрования данных, получающую на вход исходные массивы данных (обучающий и тестовый). Внутри функции:
1. С помощью np.random.normal() генерируется случайная квадратная матрица, по числу строк, равная входному массиву.
2. Матрица проверяется на обратимость. В случае необратимости, повторяется шаг 1.
3. Исходные массивы умножаются на полученную матрицу.
4. Можно выполнить масштабирование внутри функции, или передать отмасштабированные данные или опустить масштабирование (в данной задаче оно не дает профита).
5. Возвращаем зашифрованные данные и матрицу домножения (для декодирования).

**Обоснование**

Уравнение линейной регрессии после обучения на массиве данных, домноженных на обратимую матрицу не должно измениться, т.к. по сути зависимости между исходными параметрами при домножении их на некие наборы фиксированных значений создадут лишь новые вариации взаимосвязей в пропорциональном количестве, что в тоге не повлияет на регрессионную кривую. 

Доказательство отсутствия изменений представлено в п. 2. 

Функция возращвет матрицу шифровщик, чтобы при необходимости, можно было найти обратную матрицу функцией np.linalg.inv.

In [13]:
features_train.shape

(3750, 4)

## Проверка алгоритма

Создадим функцию - алгоритм шифрования.

In [14]:
def cipher(features_train, features_test):
    r = np.random.RandomState([0, 42])
    matrix_cipher = r.normal(size=(features_train.shape[1], r.randint(2, 6)))

    # вероятность получить необратимую матрицу минимальна, но выполним проверку и пересоздадим матрицу в случае неудачи
    while True:
        try:
            np.linalg.inv(matrix_cipher)
            break
        except:
            matrix_cipher = r.normal(size=(features_train.shape[1], r.randint(2, 6)))    
            
    features_train_cipher = features_train.values.dot(matrix_cipher)
    features_test_cipher = features_test.values.dot(matrix_cipher)
    
    return features_train_cipher, features_test_cipher, matrix_cipher    

Обучим модель и получим предсказания на зашифрованных данных.

In [15]:
features_train_cipher, features_test_cipher, matrix_cipher = cipher(features_train_sc, features_test_sc)

model_cipher = LinearRegression()
model_cipher.fit(features_train_cipher, target_train)
predicted_cipher = model_cipher.predict(features_test_cipher)
r2_cipher = r2_score(target_test, predicted_cipher)

print(r2_cipher)

0.4228246096919983


Обучим модель и получим предсказания на исходных данных.

In [16]:
model_raw = LinearRegression()
model_raw.fit(features_train_sc, target_train)
predicted_raw = model_raw.predict(features_test_sc)
r2_raw = r2_score(target_test, predicted_raw)

print(r2_raw)

0.42282460969199875


Проверка также показала отсутствие влияния маштабирования на показатели, в независимости от того проводится ли масштабирование до или после шифрования.

Выполним проверку работы алгоритма, осуществив расшифровку данных.

In [17]:
# найдем ключ
matrix_decipher = np.linalg.inv(matrix_cipher)

# умножим матрицу с зашифрованными данными на ключ
features_train_decipher = pd.DataFrame(features_train_cipher.dot(matrix_decipher), columns=features_train_sc.columns, index=features_train_sc.index)

# проверим результат
display(features_train_decipher.head())
display(features_train_sc.head())

,sex,age,salary,family_members
1587,-1.011263,-1.056202,-0.572663,0.728485
3380,-1.011263,0.485687,-1.668402,-0.187925
1802,0.988862,2.383397,-1.497507,-0.187925
294,0.988862,-0.581775,0.020443,0.728485
2139,0.988862,-0.581775,0.402444,-1.104336


,sex,age,salary,family_members
1587,-1.011263,-1.056202,-0.572663,0.728485
3380,-1.011263,0.485687,-1.668402,-0.187925
1802,0.988862,2.383397,-1.497507,-0.187925
294,0.988862,-0.581775,0.020443,0.728485
2139,0.988862,-0.581775,0.402444,-1.104336


Данные успешно расшифрованы.

## Вывод

С помощью применения матричных операций мы разработали алгоритм шифрования данных, не влияющий на качество обучения модели. При этом расшифровка данных без ключа (матрицы обратной матрице домножителю) представляется весьма затруднительной, т.к. даже число параметров значительно возрастает.